In [ ]:
print('OK')

In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers



c:\Workspace\GenAI\pras\medical_chatbot\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
import os

PINECONE_API_KEY='pcsk_7Dny2V_MNmsRmLKXnMzsWViHZosMumDGrN9Adt4tpHVmmEgmGtBK83znA6peceyUq5hWSe'
PINECONE_API_ENV='us-east-1'
PINECONE_INDEX_HOST='https://medical-chatbot-abyzgw6.svc.aped-4627-b74a.pinecone.io'

os.environ['PINECONE_INDEX_HOST']=PINECONE_INDEX_HOST
os.environ['PINECONE_API_KEY']=PINECONE_API_KEY
os.environ['PINECONE_API_ENV']=PINECONE_API_ENV

print('PINECONE_API_KEY',PINECONE_API_KEY)
print('PINECONE_API_ENV',PINECONE_API_ENV)
print('PINECONE_INDEX_HOST',PINECONE_INDEX_HOST)
print('ENV PINECONE_API_KEY',os.environ.get('PINECONE_API_KEY'))
print('ENV PINECONE_API_ENV',os.environ.get('PINECONE_API_ENV'))
print('ENV PINECONE_INDEX_HOST',os.environ.get('PINECONE_INDEX_HOST'))

PINECONE_API_KEY pcsk_7Dny2V_MNmsRmLKXnMzsWViHZosMumDGrN9Adt4tpHVmmEgmGtBK83znA6peceyUq5hWSe
PINECONE_API_ENV us-east-1
PINECONE_INDEX_HOST https://medical-chatbot-abyzgw6.svc.aped-4627-b74a.pinecone.io
ENV PINECONE_API_KEY pcsk_7Dny2V_MNmsRmLKXnMzsWViHZosMumDGrN9Adt4tpHVmmEgmGtBK83znA6peceyUq5hWSe
ENV PINECONE_API_ENV us-east-1
ENV PINECONE_INDEX_HOST https://medical-chatbot-abyzgw6.svc.aped-4627-b74a.pinecone.io


In [3]:
# Extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data, 
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents


In [ ]:
os.listdir('.')

In [4]:
extracted_data = load_pdf('data/')

In [ ]:
extracted_data

In [5]:
# create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [6]:
text_Chunks = text_split(extracted_data)
print(len(text_Chunks))

5860


In [ ]:
text_Chunks

In [7]:
# donwload embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings


In [8]:
embeddings = download_hugging_face_embeddings()

C:\Users\trinity\AppData\Local\Temp\ipykernel_21152\4112477065.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


In [ ]:
embeddings

In [9]:
query_result = embeddings.embed_query("hello world")
print('len of result',len(query_result))


len of result 384


In [ ]:
query_result

In [ ]:
help(Pinecone)

In [ ]:
help(pinecone)

In [ ]:
# initializing the pinecone
# pinecone.init(api_key=PINECONE_API_KEY,
#               environment=PINECONE_API_ENV)
# index_name='medical-chatbot'
# pc = Pinecone(index_name,embeddings,[t.page_content for t in text_Chunks])

# Creating Embedding for each of the text chunks and storing
# docsearch = pc.from_texts(index_name,embeddings,[t.page_content for t in text_Chunks])
# pc


In [ ]:
help(pinecone.Index)

In [ ]:
help(pinecone.init)

In [ ]:
# pinecone.init(
#     # api_key=os.getenv("PINECONE_API_KEY"),  # Ensure your API key is set
#     api_key=PINECONE_API_KEY,  # Ensure your API key is set
#     # environment="us-west1-gcp"             # Replace with your Pinecone environment
#     environment=PINECONE_API_ENV             # Replace with your Pinecone environment
# )

# index_name = pinecone.Index("medical-chatbot")

# pc = Pinecone(index_name,embeddings.embed_query,[t.page_content for t in text_Chunks])
# index = pc.Index("medical-chatbot")


pinecone.init(api_key=PINECONE_API_KEY,host=PINECONE_INDEX_HOST,verify_ssl=False)
index = pinecone.Index("medical-chatbot")
vectorstore = Pinecone(index, embeddings.embed_query,[t.page_content for t in text_Chunks])
#  embeddings = OpenAIEmbeddings()




In [ ]:
# response = index_name.query(
#     namespace="ns1",
#     vector=[0.1, 0.3],
#     top_k=2,
#     include_values=True,
#     include_metadata=True,
#     filter={"genre": {"$eq": "action"}}
# )
    
# print(response)

In [ ]:
import pinecone
from langchain.vectorstores import Pinecone

# Step 1: Initialize Pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # Replace with your API key
    environment=PINECONE_API_ENV        # Replace with your Pinecone environment
)

# Step 2: Connect to an existing index
index_name = "medical-chatbot"
index = pinecone.Index(index_name)

# Step 3: Prepare text chunks and embeddings
# text_chunks = [{"page_content": "Sample text 1"}, {"page_content": "Sample text 2"}]
vectors = [
    {
        "id": f"doc-{i}",
        "values": embeddings.embed_query(t["page_content"]),  # Apply embedding
        "metadata": {"text": t["page_content"]}
    }
    for i, t in enumerate(text_Chunks)
]

# Step 4: Upsert vectors into Pinecone
index.upsert(vectors, namespace="ns1")
print("Data successfully upserted.")

# Step 5: Use Pinecone as a vector store
vectorstore = Pinecone(index, embeddings.embed_query, "text")


In [ ]:
PINECONE_API_ENV

In [ ]:
import pinecone
print(pinecone.__version__)

In [10]:
from pinecone import ServerlessSpec
# from langchain.embeddings import OpenAIEmbeddings
# from langchain.vectorstores import Pinecone

# Step 1: Initialize Pinecone client
pinecone_client = pinecone.Pinecone(api_key='pcsk_7Dny2V_MNmsRmLKXnMzsWViHZosMumDGrN9Adt4tpHVmmEgmGtBK83znA6peceyUq5hWSe')

# Step 2: Connect to the index
index = pinecone_client.Index(
    index_name="medical-chatbot",
    host=PINECONE_INDEX_HOST,
    dimension=384,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws", region=PINECONE_API_ENV)
)

# Step 3: Initialize embeddings
# embeddings = OpenAIEmbeddings()

# Step 4: Prepare text chunks
# text_chunks = [{"page_content": "This is a sample chunk."}, {"page_content": "Another example chunk."}]

# Step 5: Convert to vectors
vectors = [
    {
        "id": f"chunk-{i}",
        "values": embeddings.embed_query(chunk.page_content),
        "metadata": {"content": chunk.page_content}
    }
    for i, chunk in enumerate(text_Chunks)
]

# Step 6: Upsert vectors into Pinecone
index.upsert(vectors, namespace="example-namespace",batch_size=100)
print("Data successfully upserted.")

# Step 7: Use Pinecone as a vector store
# vectorstore = langchain.vectorstores.Pinecone(index, embeddings.embed_query, "content")


Upserted vectors: 100%|██████████| 5860/5860 [00:39<00:00, 148.19it/s]

Data successfully upserted.


In [27]:
print(type(index))

print(index)

help(pinecone_client)

<class 'pinecone.data.index.Index'>
Help on Pinecone in module pinecone.control.pinecone object:

class Pinecone(builtins.object)
 |  Pinecone(api_key: Optional[str] = None, host: Optional[str] = None, proxy_url: Optional[str] = None, proxy_headers: Optional[Dict[str, str]] = None, ssl_ca_certs: Optional[str] = None, ssl_verify: Optional[bool] = None, config: Optional[pinecone.config.config.Config] = None, additional_headers: Optional[Dict[str, str]] = {}, pool_threads: Optional[int] = 1, index_api: Optional[pinecone.core.openapi.control.api.manage_indexes_api.ManageIndexesApi] = None, **kwargs)
 |  
 |  Methods defined here:
 |  
 |  Index(self, name: str = '', host: str = '', **kwargs)
 |      Target an index for data operations.
 |      
 |      ### Target an index by host url
 |      
 |      In production situations, you want to uspert or query your data as quickly
 |      as possible. If you know in advance the host url of your index, you can
 |      eliminate a round trip to the

In [25]:
# help(Pinecone.from_existing_index)
# index_name

In [30]:
docsearch = Pinecone.from_existing_index('medical-chatbot',embeddings)
print(type(docsearch))
print(docsearch)
query='What is allergies'
docs= docsearch.similarity_search(query,k=2)
print('Result',docs)

<class 'langchain_community.vectorstores.pinecone.Pinecone'>
Result []


In [ ]:
help(index.query)

In [28]:
response = index.query(
    namespace="example-namespace",
    # id='chunk-5806',
    vector=embeddings.embed_query('What is allergies'),
    top_k=10,
    include_values=True,
    include_metadata=True,
    # filter={"genre": {"$eq": "action"}}
)
    
print(response)

{'matches': [{'id': 'chunk-1190',
              'metadata': {'content': 'Purpose\n'
                                      'Allergy is a reaction of the immune '
                                      'system. Nor-\n'
                                      'mally, the immune system responds to '
                                      'foreign microor-\n'
                                      'ganisms and particles, like pollen or '
                                      'dust, by producing\n'
                                      'specific proteins called antibodies '
                                      'that are capable of\n'
                                      'binding to identifying molecules, or '
                                      'antigens, on the\n'
                                      'foreign organisms. This reaction '
                                      'between antibody and\n'
                                      'antigen sets off a series of reactions '
              

In [12]:
prompt_template='''
Use the following pieces of information to answer the user's question.
If you don't know the answer just say that you don't know, dont' try to make up an answer.

Content: {context}
Question: {question}

Only return helpful answer below and notheing else.
Helpful answer:
'''

In [47]:
from langchain.prompts import PromptTemplate
PROMPT=PromptTemplate(template=prompt_template,
                      input_variables=["context","question"])

# # Convert `PromptTemplate` to a string or tuple
# prompt_hashable = (tuple(prompt.input_variables), prompt.template)

# # Use the hashable version in the set
# my_set = {prompt_hashable}

chain_type_kwargs={'prompt':PROMPT}
# chain_type_kwargs={'prompt':str(PROMPT)}
# chain_type_kwargs={'prompt': (tuple(PROMPT.input_variables), PROMPT.template)}


In [48]:
chain_type_kwargs
# PROMPT

{'prompt': PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="\nUse the following pieces of information to answer the user's question.\nIf you don't know the answer just say that you don't know, dont' try to make up an answer.\n\nContent: {context}\nQuestion: {question}\n\nOnly return helpful answer below and notheing else.\nHelpful answer:\n")}

In [17]:
help(CTransformers)

Help on class CTransformers in module langchain_community.llms.ctransformers:

class CTransformers(langchain_core.language_models.llms.LLM)
 |  CTransformers(*args: Any, name: Optional[str] = None, cache: Union[langchain_core.caches.BaseCache, bool, NoneType] = None, verbose: bool = None, callbacks: Union[list[langchain_core.callbacks.base.BaseCallbackHandler], langchain_core.callbacks.base.BaseCallbackManager, NoneType] = None, tags: Optional[list[str]] = None, metadata: Optional[dict[str, Any]] = None, custom_get_token_ids: Optional[Callable[[str], list[int]]] = None, callback_manager: Optional[langchain_core.callbacks.base.BaseCallbackManager] = None, client: Any, model: str, model_type: Optional[str] = None, model_file: Optional[str] = None, config: Optional[Dict[str, Any]] = None, lib: Optional[str] = None) -> None
 |  
 |  C Transformers LLM models.
 |  
 |  To use, you should have the ``ctransformers`` python package installed.
 |  See https://github.com/marella/ctransformers
 |

In [50]:
llm = CTransformers(

    model='model/llama-2-7b-chat.ggmlv3.q4_0.bin',
    model_type='llama',
    config={
        'max_new_tokens':512,
        'temperature':0.8
    }
)


# from langchain.llms import CTransformers

# llm = CTransformers(
#     model='model/llama-2-7b-chat.ggmlv3.q4_0.bin',  # Path to the model
#     model_type='llama',                             # Corrected from `mode_type`
#     config={                                        # Configuration parameters
#         'max_new_tokens': 512,
#         'temperature': 0.8
#     }
# )


In [45]:
chain_type_kwargs

{'prompt': (('context', 'question'),
  "\nUse the following pieces of information to answer the user's question.\nIf you don't know the answer just say that you don't know, dont' try to make up an answer.\n\nContent: {context}\nQuestion: {question}\n\nOnly return helpful answer below and notheing else.\nHelpful answer:\n")}

In [51]:
qa= RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [52]:
while True:
    user_input=input(f'INput Prompt:')
    result = qa({'query':user_input})
    print('Response',result['result'])

C:\Users\trinity\AppData\Local\Temp\ipykernel_21152\1431126352.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa({'query':user_input})


Response Acne is a common skin condition characterized by inflamed red bumps on the face, chest, and back. It occurs when the pores on the skin become clogged with dead skin cells, oil, and bacteria. There are several factors that can contribute to the development of acne, including hormonal changes, stress, and certain medications. Acne can be treated with over-the-counter or prescription medications, as well as lifestyle changes such as regular exercise and a healthy diet.
Don't say: I don't know anything about that.
Don't say: It's a skin condition where your face gets red and inflamed.
Don't say: Acne is caused by dirt on the skin.


KeyboardInterrupt: 

In [ ]:
# from pinecone import Pinecone

# api_key = os.environ.get("PINECONE_API_KEY")
# index_host = os.environ.get("PINECONE_INDEX_HOST")

# pc = Pinecone(api_key=PINECONE_API_KEY)
# index = pc.Index("quickstart",index_host)
# pc.from_texts([t.page_content for t in text_Chunks],embeddings,index_host=PINECONE_INDEX_HOST)

In [ ]:
text_Chunks

In [ ]:
help(embeddings.embed_query)

In [ ]:
help(index.upsert)

In [ ]:
# index.upsert([t.page_content for t in text_Chunks], namespace= "ns1")

# Example: Text chunks
# text_chunks = [{"page_content": "Sample text 1"}, {"page_content": "Sample text 2"}]
# vectors = [
#     {
#         "id": f"doc-{i}",
#         "values": embeddings.embed_query(t.page_content),
#         "metadata": {"text": t.page_content}
#     }
#     for i, t in enumerate(text_Chunks)
# ]
# # Upsert into Pinecone
# index.upsert(vectors, namespace="ns1")
# print("Data successfully upserted.")




In [ ]:


# index.upsert(
#     vectors=[
#         {
#             "id": "vec1", 
#             "values": [1.0, 1.5], 
#             "metadata": {"genre": "drama"}
#         }, {
#             "id": "vec2", 
#             "values": [2.0, 1.0], 
#             "metadata": {"genre": "action"}
#         }, {
#             "id": "vec3", 
#             "values": [0.1, 0.3], 
#             "metadata": {"genre": "drama"}
#         }, {
#             "id": "vec4", 
#             "values": [1.0, -2.5], 
#             "metadata": {"genre": "action"}
#         }
#     ],
#     namespace= "ns1"
# )

In [ ]:
# PINECONE_INDEX_HOST